In [7]:
import pandas as pd
%load_ext line_profiler

### 와... 3시간을 뻘짓하다니.... 그래도 Dataframe에 대해서 알았으니 그것만으로 충분히 가치있었다. 

1. 쉬운 방법. drop_duplicates method 사용하기

### 혹시나 나중을 위해서 기록을 남겨둬야겠다.

**Dataframe에서 중복되는 값 제거하고 하나만 남기기 위한 방법**
1. value_counts()를 사용해 원하는 행이 중복값이 있는지 확인할 수 있음. value_counts()가 1이라면 1개만 있는 것임
   
2. pd.concat의 장점은 keys를 사용할 수 있다는 것. 매우 유용하다.
3. Series는 Boolean Mask 대신 Where이나 Mask을 사용한다. 결과를 Nan으로 반환하므로 .dropna()와 사용할 경우 원하는 index를 쉽게 얻을 수 있다.
   Where은 True인 데이터를 살리고 Mask는 False인 데이터를 살린다.
4. reset_index()는 index 순서를 초기화 할때 사용하면 편하다. drop = True, inplace = True 필수!
5. isin()은 매우 좋은 도구이다. Boolean 값을 반환하기 때문에 손쉽게 대상을 구분할 수 있음. 특히 전체 Dataframe을 대상으로 boolean column을 따로 만들어 놓으면 두 대상을 매우 쉽게 구분할 수 있음.
6. dataframe은 append가 불가능하다. list를 활용하자.
7. profiling에 대해서 배움. %load_ext line_profiler를 import 넣는 공간에 넣고  함수 만든 뒤 %lprun -f function_for_line_profiling function_for_line_profiling()을 쓰면 코드 한줄 한줄 걸리는 시간을 알 수 있다.



In [230]:
songattribute_df = pd.read_csv('data/ch12/songAttributes_1999-2019.csv')
# wsr = songattribute_df.apply(lambda r : '{}---{}'.format(r.Artist,r.Name), axis=1)
# wsr.value_counts()

In [317]:
asd = songattribute_df.drop_duplicates(['Artist','Name'])

In [232]:
artist_unique = songattribute_df.Artist.unique()

### 가수와 Name 별로 중복되는 값이 있는지 확인하기
x =[]
for poss in artist_unique :
    BM = songattribute_df.Artist == poss
    artist_df = songattribute_df[BM]
    artist_name_df = artist_df.Name.value_counts()
    x.append(artist_name_df)

In [314]:
### keys를 활용하면 보기좋게 Series를 만들 수 있다.
k = pd.concat(x,keys=artist_unique)

In [234]:
one_song = k.mask(k>1).dropna()

In [316]:
for artist in artist_unique :
    al_2 = songattribute_df[songattribute_df.Artist == artist].Name.isin(one_song[artist].index)
    songattribute_df.loc[al_2.index,'new_name'] = al_2



#### 위 방법은 2초정도 느림.. 별로 차이안나네 한번에 리스트로 만든 뒤 append 하면 됨

# qwe =[]
# for artist in artist_unique :
#     al_2 = songattribute_df[songattribute_df.Artist == artist].Name.isin(one_song[artist].index)
#     qwe.append(al_2)

# qwe
# # songattribute_df[['Name','new_name']]

In [320]:
### single attributes
sin_finish = songattribute_df[songattribute_df['new_name'] == True]

In [256]:
### Mask를 쓰면 매우매우 손쉽게 원하는  대상을 구분할 수 있다.
dup_song = k.mask(k==1).dropna()
dup_song

10 Years  Insomnia            2.0
          Novacaine           2.0
          The Messenger       2.0
          Phantoms            2.0
          Vampires            2.0
                             ... 
t.A.T.u.  Gomenasai           2.0
          Friend or Foe       2.0
          Not Gonna Get Us    2.0
          How Soon Is Now?    2.0
          Objzienka Nol       2.0
Name: Name, Length: 17193, dtype: float64

In [318]:
dup = songattribute_df[songattribute_df['new_name'] == False]

asd = pd.DataFrame(columns=songattribute_df.columns)
# asd = []

for artist in dup.Artist.unique() :
    for song in dup_song[artist].index :
        a = dup.Name.where(dup.Name == song).dropna().index[0]
        asd.append(dup.loc[a])

In [296]:
dup_finish = pd.DataFrame(asd)


In [321]:
dup_finish.reset_index(inplace=True, drop=True)

In [309]:
### Single과 duplicate를 합친다.
final_attribute = pd.concat([sin_finish,dup_finish])
final_attribute.sort_values(by='Artist',inplace=True)
final_attribute.reset_index(inplace=True, drop=True)

final_attribute

,Unnamed: 0,Acousticness,Album,Artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Name,Popularity,Speechiness,Tempo,TimeSignature,Valence,new_name
0,55,0.000048,Division,10 Years,0.492,203333,0.915,False,0.000181,0.5090,-2.834,0,Actions & Motives,8,0.0377,150.951,4,0.5540,True
1,81,0.408000,Live & Unplugged At The Tennessee Theatre,10 Years,0.236,259726,0.482,False,0.000015,0.9630,-8.621,0,All Your Lies - Live,19,0.0448,49.020,4,0.3330,True
2,80,0.754000,Live & Unplugged At The Tennessee Theatre,10 Years,0.297,363422,0.403,False,0.316000,0.6800,-9.862,1,Focus - Live,20,0.0297,101.363,4,0.0368,True
3,79,0.950000,From Birth to Burial,10 Years,0.234,250893,0.410,False,0.744000,0.1190,-9.467,1,Moisture Residue,28,0.0325,84.779,4,0.1390,True
4,78,0.000848,From Birth to Burial,10 Years,0.331,206480,0.892,False,0.689000,0.2170,-6.450,1,Miscellanea,33,0.0583,169.890,4,0.5210,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131372,14,0.194000,#willpower (Deluxe),will.i.am,0.728,232667,0.804,False,0.000000,0.0984,-5.491,1,Ghetto Ghetto,27,0.3290,170.038,4,0.3470,True
131373,13,0.011600,#willpower (Deluxe),will.i.am,0.677,233947,0.757,False,0.000000,0.1260,-3.398,1,Far Away From Home,31,0.0334,116.030,4,0.8210,True
131374,12,0.135000,#willpower (Deluxe),will.i.am,0.517,251027,0.597,False,0.000000,0.4050,-5.917,0,Love Bullets,30,0.0353,129.976,4,0.1910,True
131375,22,0.154000,Lost Change,will.i.am,0.906,228027,0.652,True,0.000000,0.1610,-4.696,0,If You Didn't Know,8,0.4000,96.024,4,0.7820,True


In [16]:
%lprun -f function_for_line_profiling function_for_line_profiling()

Timer unit: 1e-07 s

Total time: 4.99052 s
File: <ipython-input-15-783af6ae6475>
Function: function_for_line_profiling at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def function_for_line_profiling() :
     2         2        200.0    100.0      0.0      for i , artist in enumerate(artist_unique[:1]) :
     3       131      10341.0     78.9      0.0          for a in x[i] :
     4       130   23526352.0 180971.9     47.1              BM = (songattribute_df.Artist == artist) & (songattribute_df.Name == a)
     5       130    1026786.0   7898.4      2.1              remove_index = songattribute_df[BM].index[1:]
     6       130   25341519.0 194934.8     50.8              songattribute_df.drop(index=remove_index, inplace=True)